# Trabajo final data mining

En este trabajo brindaremos un chat bot de respuestas cortas para IM en español. Primero que nada definiremos una respuesta corta como cualquier turno hasta 3 palabras que ocurren al menos n veces. Iremos probando diferentes n a modo de prueba. El texto anterior a la respuesta corta será llamado contexto

## Extracción del corpus

Debido a que no hay un corpus bien definido para este problema utilizaremos el contexto del chat como texto de entrenamiento, por lo que la respuesta corta será la etiqueta. Utilizaremos mensajes de whatsapp tomadas de nuestras conversaciones.

### Procesamiento

In [1]:
import re
import csv
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, timedelta
from string import punctuation
from collections import defaultdict

In [2]:
files = glob.glob('todos/*')
raw_data = []
for file in files:
    with open(file, 'r') as f:
        raw_data += [f.readlines()]

Debemos quitar líneas que no son parte de los chats. Todos contienen fecha y hora.

`6/18/17, 16:43 - Los mensajes y llamadas en este chat ahora están protegidos con cifrado de extremo a extremo. Toca para más información.`

`3/22/17, 19:12 - Mati created group "Discreta II"\n`

`4/6/15, 22:44 - You were added\n`

`2/1/17, 01:41 - NN's security code changed. Tap for more info.\n`

In [7]:
raw_data[0]

['10/16/17, 17:44 - Messages to this chat and calls are now secured with end-to-end encryption. Tap for more info.\n',
 '10/16/17, 17:44 - \u202a+54 9 351 328-5957\u202c: https://m.facebook.com/story.php?story_fbid=1490068964433492&id=161872773919791\n',
 '10/16/17, 17:54 - Daniela Bosch: Hola Te confundiste de numero\n']

Parte del mismo mensaje :o. Como hacemos? Deberíamos buscar una alternativa

In [8]:
# raw_data[18129] + raw_data[18130] + raw_data[18131]
raw_data[1][1:6]

['29/9/17, 16:59 - Ori creó el grupo "Libertad 🗽 🌏"\n',
 '6/10/17, 16:57 - Ori te añadió\n',
 '6/10/17, 16:57 - Ahora eres un administrador\n',
 '6/10/17, 16:58 - Ori: 😂😂😂😂😂😂😂😂😂😂\n',
 '6/10/17, 17:43 - Luciano: Que es este lugar 👀\n']

#### Formato lindo

Para ello spliteo cada línea por la primera ocurrencia de ": ". Ver bien si se puede hacer algo con las líneas que no tengan el mismo formato (por ahora la quitamos). Hay un problema con las fechas debido a la configuración de cada telefono. Algunos formatos es mm/dd/yyyy y otros dd/mm/yyyy. Veremos como solucionarlo en un futuro.

In [23]:
def is_message(text):
    """
    No es un mensaje de varias líneas.
    """
    return re.search(r'\d+/\d+/\d+, \d+:\d+.+ - .+: ', text) is not None
    # return ': ' in text and '/' in text and ',' in text

def is_continuation(text):
    """
    Pertenece al mismo mensaje anterior.
    """
    return re.search(r'\d+/\d+/\d+, \d+:\d+.+ - ', text) is None
    # return '/' not in text or ': ' not in text or ',' not in text

def is_meta(text):
    """
    Contiene metadatos sobre el chat.
    """
    return re.search(r'\d+/\d+/\d+, \d+:\d+.+ - ', text) is not None and ': ' not in text

# at the begining we don't filter any kind of message. Let's see what happend.
info_chat = []
for i, chat in enumerate(raw_data):
    users = {}
    for line in chat:
        line = line.strip('\n')
        if is_meta(line):
            continue
        if is_continuation(line):
            info_chat[len(info_chat)-1][4] += ". " + line
            continue
        if is_message(line):
            info, text = line.split(": ", 1)
            datetime, owner = info.split(" - ", 1)
            date, time = datetime.split(", ", 1)
            
            # anonymizing users
            if owner not in users:
                users[owner] = str(len(users))
            user = "USER" + users[owner]
                
            info_chat.append([i, user, date, time, text])

Ahora guardaremos el corpus en formato csv para hacerlo legible. 

Tendrá cinco columnas: numero de chat, persona que escribe el msj, fecha y hora en que se envío, mensaje.

| idchat ||  Owner  ||     Date    || hour ||    Message     |
|--------||---------||-------------||------||----------------|
|    1   ||  USER0  || 20-05-2017  || 20:10|| hola que haces |
|    1   ||  USER1  || 21-05-2017  || 12:20|| Todo bien?     |
|    2   ||  USER0  || 15-07-2017  || 13:15||Me fui al baile |

In [24]:
info_chat[:10]

[[0,
  'USER0',
  '10/16/17',
  '17:44',
  'https://m.facebook.com/story.php?story_fbid=1490068964433492&id=161872773919791'],
 [0, 'USER1', '10/16/17', '17:54', 'Hola Te confundiste de numero'],
 [1, 'USER0', '6/10/17', '16:58', '😂😂😂😂😂😂😂😂😂😂'],
 [1, 'USER1', '6/10/17', '17:43', 'Que es este lugar 👀'],
 [1, 'USER2', '6/10/17', '17:44', 'Qur hace este aqui'],
 [1, 'USER2', '6/10/17', '17:44', '\U0001f923'],
 [1, 'USER3', '6/10/17', '17:44', 'Bienvenido'],
 [1, 'USER1', '6/10/17', '17:50', 'Vengo a pasarle fotos obscenas e irme ?)'],
 [1, 'USER1', '6/10/17', '17:50', 'Pasarles'],
 [1, 'USER3', '6/10/17', '17:51', '👀']]

In [25]:
fieldnames = ['id', 'owner', 'date', 'hour', 'message']
with open('corpus.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    
    for idc, owner, date, hour, message in info_chat:
        if message =='':
            continue
        writer.writerow({'id': idc, 'owner': owner, 'date': date, 'hour': hour, 'message': message})